In [ ]:
from conllu import parse

with open('data/russian.conllu', 'r', encoding='utf8') as file:
    data = file.read()

sentences = parse(data)

import spacy
from spacy.tokens import Doc, Token
from conllu import parse_incr
from conllu import parse
from spacy.vocab import Vocab
from spacy import displacy
from pprint import pprint

nlp = spacy.blank("ru")  
vocab = nlp.vocab

if not Token.has_extension("xpos"):
    Token.set_extension("xpos", default=None)
if not Token.has_extension("deps"):
    Token.set_extension("deps", default=None)
if not Token.has_extension("misc"):
    Token.set_extension("misc", default=None)


def process_conllu_file(file_path):
    docs = []
    with open(file_path, "r", encoding="utf-8") as file:
        for conllu_sentence in parse_incr(file):
            doc = conllu_to_spacy_doc(conllu_sentence, vocab)
            docs.append(doc)
    return docs

def conllu_to_spacy_doc(conllu_sentence, vocab):
    words = [token["form"] for token in conllu_sentence]
    heads = [token["head"] if token["head"] is not None else 0 for token in conllu_sentence]  # Head = 0 para raíces
    deps = [token["deprel"].split(":")[0] for token in conllu_sentence]
    lemmas = [token["lemma"] for token in conllu_sentence]
    pos_tags = [token["upostag"] for token in conllu_sentence]
    xpos_tags = [token["xpos"] for token in conllu_sentence]
    morph_feats = [token["feats"] or {} for token in conllu_sentence]
    misc_info = [token["misc"] or {} for token in conllu_sentence]
    deps_info = [token["deps"] or {} for token in conllu_sentence]

    # Crear el objeto Doc
    doc = Doc(vocab, words=words)

    # Asignar los atributos a cada token
    for i, token in enumerate(doc):
        token.lemma_ = lemmas[i]
        
        # Asignar el Universal POS a `pos_`, o usar `tag_` si no es válido
        if pos_tags[i] != "_":
            token.pos_ = pos_tags[i]
        elif xpos_tags[i] is not None:  # Verificar si xpos_tags[i] no es None antes de asignarlo
            token.tag_ = xpos_tags[i]  # Usar `xpos` como `tag_` si `upos` es "_"

        token.dep_ = deps[i]
        
        # Verificar que heads[i] sea un número entero antes de asignarlo como cabeza
        if isinstance(heads[i], int) and heads[i] > 0:
            token.head = doc[heads[i] - 1]  # Usar head como índice (spaCy usa índice 0 para la raíz)
        else:
            token.head = token  # Si head es 0 o no válido, el token se apunta a sí mismo como raíz

        # Asignar xpos, deps y misc como atributos personalizados
        token._.xpos = xpos_tags[i]
        token._.deps = deps_info[i]
        token._.misc = misc_info[i]

        # Convertir `feats` a un formato que spaCy entiende y asignarlo a morph
        if morph_feats[i]:
            feats_str = "|".join(f"{k}={v}" for k, v in morph_feats[i].items())
            token.set_morph(feats_str)

    return doc

def render_doc_by_sent_id(docs, sentences, target_sent_id):
    # Buscar el índice del `sent_id` deseado en los metadatos de `sentences`
    for i, sentence in enumerate(sentences):
        if sentence.metadata.get("sent_id") == target_sent_id:
            # Renderizar el Doc correspondiente usando `displacy`
            displacy.render(docs[i], style="dep", jupyter=True)
            return  # Salimos después de encontrar y renderizar
    print(f"Oración con sent_id '{target_sent_id}' no encontrada.")


file_path = "data/russian.conllu"
docs = process_conllu_file(file_path)




PLACE

In [19]:
render_doc_by_sent_id(docs, sentences, '1')
# The cat is in the wardrobe. [place, IN]


print(sentences[0])

for token in sentences[0]:
    print(token.items())

TokenList<Кошка, в, шкафу, ., metadata={generator: "UDPipe 2, https://lindat.mff.cuni.cz/services/udpipe", udpipe_model: "russian-syntagrus-ud-2.12-230717", udpipe_model_licence: "CC BY-NC-SA", newdoc: "None", newpar: "None", sent_id: "1", text: "Кошка в шкафу."}>
dict_items([('id', 1), ('form', 'Кошка'), ('lemma', 'кошка'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '0:5'})])
dict_items([('id', 2), ('form', 'в'), ('lemma', 'в'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 3), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '6:7'})])
dict_items([('id', 3), ('form', 'шкафу'), ('lemma', 'шкаф'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan', 'Case': 'Loc', 'Gender': 'Masc', 'Number': 'Sing'}), ('head', 1), ('deprel', 'nmod'), ('deps', None), ('misc', {'SpaceAfter': 'No', 'TokenRange': '8:13'})])
dict_

In [ ]:
sentences[0][2].get('feats').get('Case')

'Loc'

In [20]:
render_doc_by_sent_id(docs, sentences, '4')
# The book is on the table. [place, ON]

print(sentences[3])

for token in sentences[3]:
    print(token.items())



TokenList<Книга, на, столе, ., metadata={sent_id: "4", text: "Книга на столе."}>
dict_items([('id', 1), ('form', 'Книга'), ('lemma', 'книга'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '57:62'})])
dict_items([('id', 2), ('form', 'на'), ('lemma', 'на'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 3), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '63:65'})])
dict_items([('id', 3), ('form', 'столе'), ('lemma', 'стол'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan', 'Case': 'Loc', 'Gender': 'Masc', 'Number': 'Sing'}), ('head', 1), ('deprel', 'nmod'), ('deps', None), ('misc', {'SpaceAfter': 'No', 'TokenRange': '66:71'})])
dict_items([('id', 4), ('form', '.'), ('lemma', '.'), ('upos', 'PUNCT'), ('xpos', None), ('feats', None), ('head', 1), ('deprel', 'punct'), ('deps', None), ('misc', {'SpacesAfter': '

In [ ]:
render_doc_by_sent_id(docs, sentences, '7')
# The bird rests next to the tree. [place, AT]

print(sentences[6])

for token in sentences[6]:
    print(token.items())

TokenList<Птица, отдыхает, у, дерева, ., metadata={sent_id: "7", text: "Птица отдыхает у дерева."}>
dict_items([('id', 1), ('form', 'Птица'), ('lemma', 'птица'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '123:128'})])
dict_items([('id', 2), ('form', 'отдыхает'), ('lemma', 'отдыхать'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Aspect': 'Imp', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '129:137'})])
dict_items([('id', 3), ('form', 'у'), ('lemma', 'у'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '138:139'})])
dict_items([('id', 4), ('form', 'дерева'), ('lemma', 'дерево'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Ina

GOAL

In [22]:
render_doc_by_sent_id(docs, sentences, '2')
# She enters the shop.

print(sentences[1])

for token in sentences[1]:
    print(token.items())

TokenList<Она, входит, в, магазин, ., metadata={sent_id: "2", text: "Она входит в магазин."}>
dict_items([('id', 1), ('form', 'Она'), ('lemma', 'она'), ('upos', 'PRON'), ('xpos', None), ('feats', {'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing', 'Person': '3', 'PronType': 'Prs'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '15:18'})])
dict_items([('id', 2), ('form', 'входит'), ('lemma', 'входить'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Aspect': 'Imp', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '19:25'})])
dict_items([('id', 3), ('form', 'в'), ('lemma', 'в'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '26:27'})])
dict_items([('id', 4), ('form', 'магазин'), ('lemma', 'магазин'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan'

In [23]:
render_doc_by_sent_id(docs, sentences, '5')
# The boy jumped onto the table.

print(sentences[4])

for token in sentences[4]:
    print(token.items())

TokenList<Мальчик, прыгнул, на, стол, ., metadata={sent_id: "5", text: "Мальчик прыгнул на стол."}>
dict_items([('id', 1), ('form', 'Мальчик'), ('lemma', 'мальчик'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '73:80'})])
dict_items([('id', 2), ('form', 'прыгнул'), ('lemma', 'прыгать'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Aspect': 'Perf', 'Gender': 'Masc', 'Mood': 'Ind', 'Number': 'Sing', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '81:88'})])
dict_items([('id', 3), ('form', 'на'), ('lemma', 'на'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '89:91'})])
dict_items([('id', 4), ('form', 'стол'), ('lemma', 'стол'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan

In [ ]:
render_doc_by_sent_id(docs, sentences, '8')
# The boy runs toward the tree.


print(sentences[7])

for token in sentences[7]:
    print(token.items())

TokenList<Мальчик, бежит, к, дереву, ., metadata={sent_id: "8", text: "Мальчик бежит к дереву."}>
dict_items([('id', 1), ('form', 'Мальчик'), ('lemma', 'мальчик'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '148:155'})])
dict_items([('id', 2), ('form', 'бежит'), ('lemma', 'бежать'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Aspect': 'Imp', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '156:161'})])
dict_items([('id', 3), ('form', 'к'), ('lemma', 'к'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '162:163'})])
dict_items([('id', 4), ('form', 'дереву'), ('lemma', 'дерево'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan'

SOURCE

In [26]:

render_doc_by_sent_id(docs, sentences, '3')
# He came out of the cave.


print(sentences[2])

for token in sentences[2]:
    print(token.items())

TokenList<Он, вышел, из, пещеры, ., metadata={sent_id: "3", text: "Он вышел из пещеры."}>
dict_items([('id', 1), ('form', 'Он'), ('lemma', 'он'), ('upos', 'PRON'), ('xpos', None), ('feats', {'Case': 'Nom', 'Gender': 'Masc', 'Number': 'Sing', 'Person': '3'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '37:39'})])
dict_items([('id', 2), ('form', 'вышел'), ('lemma', 'выйти'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Aspect': 'Perf', 'Gender': 'Masc', 'Mood': 'Ind', 'Number': 'Sing', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '40:45'})])
dict_items([('id', 3), ('form', 'из'), ('lemma', 'из'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '46:48'})])
dict_items([('id', 4), ('form', 'пещеры'), ('lemma', 'пещера'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan', 'Case': 'Gen', 'Gende

In [29]:
render_doc_by_sent_id(docs, sentences, '6')
# The girl climbed down from the table.


print(sentences[5])

for token in sentences[5]:
    print(token.items())

TokenList<Девочка, слезла, со, стола, ., metadata={sent_id: "6", text: "Девочка слезла со стола."}>
dict_items([('id', 1), ('form', 'Девочка'), ('lemma', 'девочка'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Anim', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '98:105'})])
dict_items([('id', 2), ('form', 'слезла'), ('lemma', 'слезть'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Aspect': 'Perf', 'Gender': 'Fem', 'Mood': 'Ind', 'Number': 'Sing', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '106:112'})])
dict_items([('id', 3), ('form', 'со'), ('lemma', 'с'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '113:115'})])
dict_items([('id', 4), ('form', 'стола'), ('lemma', 'стол'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Ina

In [30]:
render_doc_by_sent_id(docs, sentences, '9')
#The squirrel moved away from the tree.

print(sentences[8])

for token in sentences[8]:
    print(token.items())




TokenList<Белка, отошла, от, дерева, ., metadata={sent_id: "9", text: "Белка отошла от дерева."}>
dict_items([('id', 1), ('form', 'Белка'), ('lemma', 'белка'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}), ('head', 2), ('deprel', 'nsubj'), ('deps', None), ('misc', {'TokenRange': '172:177'})])
dict_items([('id', 2), ('form', 'отошла'), ('lemma', 'отойти'), ('upos', 'VERB'), ('xpos', None), ('feats', {'Aspect': 'Perf', 'Gender': 'Fem', 'Mood': 'Ind', 'Number': 'Sing', 'Tense': 'Past', 'VerbForm': 'Fin', 'Voice': 'Act'}), ('head', 0), ('deprel', 'root'), ('deps', None), ('misc', {'TokenRange': '178:184'})])
dict_items([('id', 3), ('form', 'от'), ('lemma', 'от'), ('upos', 'ADP'), ('xpos', None), ('feats', None), ('head', 4), ('deprel', 'case'), ('deps', None), ('misc', {'TokenRange': '185:187'})])
dict_items([('id', 4), ('form', 'дерева'), ('lemma', 'дерево'), ('upos', 'NOUN'), ('xpos', None), ('feats', {'Animacy': 'Inan